# Install important libraries

In [1]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 16.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 46.4 MB/s eta 0:00:00


In [2]:
!pip install --upgrade accelerate -q
!pip uninstall -y transformers accelerate -q
!pip install transformers accelerate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 29.6 MB/s eta 0:00:00


# mount drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import All important libraries

In [36]:
import pandas as pd
from datasets import Dataset, DatasetDict
from datasets import load_dataset, load_from_disk, load_metric
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Extract zip file

In [37]:
from zipfile import ZipFile
file_name="/content/drive/MyDrive/archive (24).zip"
with ZipFile(file_name,'r')as zip:
  zip.extractall()
  print('Done')

Done


# read csv file

In [38]:
data = pd.read_csv("/content/Resume/Resume.csv")

In [39]:
type(data)

pandas.core.frame.DataFrame

# first 5 value

In [40]:
data.head()

,ID,Resume_str,Resume_html,Category
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR
2,33176873,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR
3,27018550,HR SPECIALIST Summary Dedica...,"<div class=""fontsize fontface vmargins hmargin...",HR
4,17812897,HR MANAGER Skill Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",HR


# shape of dataset

In [41]:
data.shape

(2484, 4)

# unique of category

In [42]:
num_labels = data['Category'].nunique()
num_labels

24

# sample of data

In [43]:
data.sample(5)

,ID,Resume_str,Resume_html,Category
785,35422305,"NATIONAL ADMINISTRATOR, POPULATION HE...","<div class=""fontsize fontface vmargins hmargin...",HEALTHCARE
1933,26509539,CONSTRUCTION INSTALLER Summary ...,"<div class=""fontsize fontface vmargins hmargin...",CONSTRUCTION
2143,22754014,CONTENT STRATEGIST Summary ...,"<div class=""fontsize fontface vmargins hmargin...",PUBLIC-RELATIONS
798,10816645,GROUP FITNESS COORDINATOR Profe...,"<div class=""fontsize fontface vmargins hmargin...",FITNESS
1996,27243670,SENIOR CONSTRUCTION & SAFETY PROJECT ...,"<div class=""fontsize fontface vmargins hmargin...",CONSTRUCTION


# train test split

In [44]:
from sklearn.model_selection import train_test_split

# rename the columns

In [45]:
data = data[['Resume_str', 'Category']].rename(columns={'Resume_str': 'resume_str', 'Category': 'category'})

# Split the data into training and temporary datasets

In [46]:
train_data, temp_data = train_test_split(data, test_size=0.2, random_state=42)


In [47]:
test_data, validation_data = train_test_split(temp_data, test_size=0.5, random_state=42)

# Create datasets from pandas DataFrames

In [48]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)
validation_dataset = Dataset.from_pandas(validation_data)

In [75]:
#train_dataset[0]

# train test validation

In [50]:
from datasets import DatasetDict

dataset_dict = DatasetDict({
    'train': train_dataset,
    'test': test_dataset,
    'validation': validation_dataset
})

In [51]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['resume_str', 'category', '__index_level_0__'],
        num_rows: 1987
    })
    test: Dataset({
        features: ['resume_str', 'category', '__index_level_0__'],
        num_rows: 248
    })
    validation: Dataset({
        features: ['resume_str', 'category', '__index_level_0__'],
        num_rows: 249
    })
})

# install hugging face

In [52]:
!pip install huggingface_hub -q

In [54]:
!huggingface-cli login --token $HUGGINGFACE_TOKEN

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [55]:
from datasets import DatasetDict

# push to hugging face

In [56]:
dataset_dict.push_to_hub('resume_dataset1')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/578 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/adas100/resume_dataset1/commit/61d41264ed1a280785aa4d39d197594c6bafed44', commit_message='Upload dataset', commit_description='', oid='61d41264ed1a280785aa4d39d197594c6bafed44', pr_url=None, pr_revision=None, pr_num=None)

# load dataset

In [59]:
data = load_dataset("adas100/resume_dataset1")
data


Generating train split:   0%|          | 0/1987 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/248 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/249 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['resume_str', 'category', '__index_level_0__'],
        num_rows: 1987
    })
    test: Dataset({
        features: ['resume_str', 'category', '__index_level_0__'],
        num_rows: 248
    })
    validation: Dataset({
        features: ['resume_str', 'category', '__index_level_0__'],
        num_rows: 249
    })
})

# tokenize model

In [60]:
model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

# Create label mappings

In [61]:

unique_labels = set(data['train']['category'] + data['test']['category'] + data['validation']['category'])
label2id = {label: id for id, label in enumerate(unique_labels)}
id2label = {id: label for label, id in label2id.items()}

#Tokenize the dataset

In [62]:
def tokenize_and_encode(ds):
    tokenized = tokenizer(ds['resume_str'], padding='max_length', truncation=True, max_length=512)
    tokenized['labels'] = [label2id[label] for label in ds['category']]
    return tokenized

In [63]:
tokenized_datasets = data.map(tokenize_and_encode, batched=True)

Map:   0%|          | 0/1987 [00:00<?, ? examples/s]

Map:   0%|          | 0/248 [00:00<?, ? examples/s]

Map:   0%|          | 0/249 [00:00<?, ? examples/s]

In [64]:
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=len(unique_labels), id2label=id2label, label2id=label2id)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Pass training arguments

In [70]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
)

# pre,recall,f1 matrix computation

In [71]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall}

# training dataset

In [72]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.697236,0.831325,0.790735,0.790828,0.803806
2,1.101800,0.632771,0.847390,0.802663,0.803538,0.810786
3,0.555000,0.648451,0.859438,0.825700,0.842987,0.821862
4,0.414500,0.644909,0.859438,0.818939,0.819889,0.828663
5,0.299400,0.644327,0.863454,0.839043,0.846241,0.843156


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=2485, training_loss=0.5260236621142873, metrics={'train_runtime': 580.279, 'train_samples_per_second': 17.121, 'train_steps_per_second': 4.282, 'total_flos': 1316579948421120.0, 'train_loss': 0.5260236621142873, 'epoch': 5.0})

# Save the model and tokenizer

In [73]:
model.save_pretrained('./fine_tuned_model')
tokenizer.save_pretrained('./fine_tuned_model_tokens')

('./fine_tuned_model_tokens/tokenizer_config.json',
 './fine_tuned_model_tokens/special_tokens_map.json',
 './fine_tuned_model_tokens/vocab.txt',
 './fine_tuned_model_tokens/added_tokens.json',
 './fine_tuned_model_tokens/tokenizer.json')

# Evaluate the model

In [74]:
eval_results = trainer.evaluate()
eval_results

{'eval_loss': 0.6443271636962891,
 'eval_accuracy': 0.8634538152610441,
 'eval_f1': 0.8390428385126438,
 'eval_precision': 0.8462414397197007,
 'eval_recall': 0.8431556969600449,
 'eval_runtime': 4.4314,
 'eval_samples_per_second': 56.189,
 'eval_steps_per_second': 14.217,
 'epoch': 5.0}

# accuracy is 86 % after 5 epoch